In [1]:
import pypyodbc
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.spatial.distance import cdist

from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard
from sqlalchemy import create_engine
import urllib

In [14]:
df_1 = pd.read_csv("all_reviews_batch_0.csv")
df_2 = pd.read_csv("all_reviews_batch_1.csv")
df_3 = pd.read_csv("all_reviews_batch_2.csv")
df_4 = pd.read_csv("all_reviews_batch_3.csv")
df = pd.concat([df_1,df_2,df_3,df_4]).reset_index(drop=True)

In [15]:
df.head(20)

,imdb_id,id,author,authorUrl,date,content,heading,stars,review_votes,review_votes_as_helpful,review_link
0,tt0765010,rw8870065,ambusched94,https://www.imdb.com/user/ur149648255/?ref_=tt...,13 February 2023,"Tobey Maguire is captivating as the lead, prov...",Tobey Maguire is splendid in a serious drama l...,5,0,0,https://www.imdb.com/review/rw8870065
1,tt0765010,rw8867540,RosanaBotafogo,https://www.imdb.com/user/ur82872124/?ref_=tt_urv,12 February 2023,"Sam is married to Grace, the father of two gir...",I loved it...,9,0,0,https://www.imdb.com/review/rw8867540
2,tt0765010,rw8866080,sidelsworthy,https://www.imdb.com/user/ur161017734/?ref_=tt...,11 February 2023,From the opening to the ending of this film. I...,Very powerful,10,0,0,https://www.imdb.com/review/rw8866080
3,tt0765010,rw8856480,maciej-slo,https://www.imdb.com/user/ur47044797/?ref_=tt_urv,6 February 2023,"This movie was very well acted, performances d...","flawlessly acted cliche, no resolution ending",6,0,0,https://www.imdb.com/review/rw8856480
4,tt0765010,rw8830329,verticalhorizonation,https://www.imdb.com/user/ur69322024/?ref_=tt_urv,27 January 2023,This film was able to take a certain human sit...,I did not feel the 100 min go by,8,0,0,https://www.imdb.com/review/rw8830329
5,tt0765010,rw8721592,M7SH,https://www.imdb.com/user/ur50675969/?ref_=tt_urv,7 December 2022,"When writing a story, sometimes a person gets ...",Ideas can't help themselves,1,4,0,https://www.imdb.com/review/rw8721592
6,tt0765010,rw8636287,cashbacher,https://www.imdb.com/user/ur97480288/?ref_=tt_urv,25 October 2022,This movie is a powerful tale that has no happ...,Brutality witnessed makes deep changes in people,10,5,3,https://www.imdb.com/review/rw8636287
7,tt0765010,rw8619612,tadarahul,https://www.imdb.com/user/ur157846883/?ref_=tt...,18 October 2022,I got this movie recommend via reels.And it wa...,Rollercoaster of emotions,9,1,1,https://www.imdb.com/review/rw8619612
8,tt0765010,rw8525305,ab_the_critic,https://www.imdb.com/user/ur11865546/?ref_=tt_urv,8 September 2022,Why folks if USA believe that the they are cen...,A Country of Entitled Military folks n Entitle...,4,14,3,https://www.imdb.com/review/rw8525305
9,tt0765010,rw8442094,sergiodarocha,https://www.imdb.com/user/ur106872206/?ref_=tt...,13 August 2022,I just saw half of this movie an week ago (08 ...,How a bad actor can ruin a movie....,5,11,1,https://www.imdb.com/review/rw8442094


Ta chỉ cần tập trung vào 3 cột nhất định để xây dựng thuật toán đề xuất là mã định danh bộ phim và người xem tương ứng với số điểm rating (stars).

In [16]:
movie_id = "imdb_id"
author_username = "author"
ratings = "stars"

In [17]:
df_filter = df[["date",movie_id,author_username,ratings]]
df_filter.drop_duplicates(inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_14476\1424583233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter.drop_duplicates(inplace=True)


In [18]:
# Filter users that watched more than 10 movies only
users_count = df_filter.groupby(["author"])['imdb_id'].count().reset_index(name='count').sort_values(['count'],ascending=False)
users_count = users_count[users_count['count'] > 10]
# Retrieve those users
author_verified = users_count['author'].tolist()

df_filter = df_filter[df_filter['author'].isin(author_verified)]

# Filter movies whose viewers has more than 50
movies_count = df_filter.groupby(["imdb_id"])['imdb_id'].count().reset_index(name='count').sort_values(['count'],ascending=False)
movies_count = movies_count[movies_count['count'] > 50]
movies_verified = movies_count['imdb_id'].tolist()

df_filter = df_filter[df_filter['imdb_id'].isin(movies_verified)]

In [19]:
df_matrix = df_filter.pivot_table(index=author_username, columns=movie_id, values=ratings)
df_matrix.replace(np.nan,0,inplace=True)
df_matrix.reset_index(inplace=True)
df_matrix

imdb_id,author,tt0000417,tt0010323,tt0013442,tt0015648,tt0015864,tt0017136,tt0020629,tt0021749,tt0021814,...,tt9772374,tt9777666,tt9783600,tt9784456,tt9784798,tt9794630,tt9844522,tt9845564,tt9848626,tt9907782
0,007Waffles,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00Yasser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,04GreatFlick,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0maro0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0w0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12532,zzapper-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12533,zzhhhqing,0.0,0.0,0.0,0.0,10.0,9.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12534,zzyzx000-124-271558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12535,~AleXa~,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_filter['date'] = pd.to_datetime(df_filter['date'])
df_filter.reset_index(drop=True,inplace=True)

In [21]:
df_filter

,date,imdb_id,author,stars
0,2023-02-13,tt0765010,ambusched94,5
1,2023-02-12,tt0765010,RosanaBotafogo,9
2,2022-10-25,tt0765010,cashbacher,10
3,2022-07-21,tt0765010,nicktusk-95591,7
4,2022-05-01,tt0765010,peterwixongb,7
...,...,...,...,...
450929,2006-12-28,tt0454082,superduperspit,5
450930,2006-12-26,tt0454082,FrightMeter,9
450931,2006-12-25,tt0454082,sackjigler,7
450932,2006-12-25,tt0454082,Nightman85,3


Ta chia tập training và testing theo tiêu chí như sau:
- Tập training sẽ lấy 8 phần ưu tiên các ngày mà user rating cũ nhất để làm dữ liệu training.
- Tập test sẽ lấy 2 phần ưu tiên các ngày mới nhất để dự đoán rating.

In [22]:
users_filter = df_filter['author'].unique()
test_index = []
for user in users_filter:
    currentUser_rating = df_filter[df_filter['author'] == user].sort_values(by='date',ascending=False)
    testing_portion = int(len(currentUser_rating) * 0.2)
    testing_set = currentUser_rating.head(testing_portion)
    testing_set_index = testing_set.index.values.tolist()
    test_index.extend([k for k in testing_set_index])

In [23]:
test_df = df_filter.iloc[test_index]
train_df = df_filter.drop(index=test_index)

In [26]:
df_filter

,date,imdb_id,author,stars
0,2023-02-13,tt0765010,ambusched94,5
1,2023-02-12,tt0765010,RosanaBotafogo,9
2,2022-10-25,tt0765010,cashbacher,10
3,2022-07-21,tt0765010,nicktusk-95591,7
4,2022-05-01,tt0765010,peterwixongb,7
...,...,...,...,...
450929,2006-12-28,tt0454082,superduperspit,5
450930,2006-12-26,tt0454082,FrightMeter,9
450931,2006-12-25,tt0454082,sackjigler,7
450932,2006-12-25,tt0454082,Nightman85,3


In [24]:
test_df

,date,imdb_id,author,stars
44162,2023-03-12,tt0167261,ambusched94,8
77979,2023-03-07,tt0472043,ambusched94,8
0,2023-02-13,tt0765010,ambusched94,5
166439,2023-02-12,tt0112642,ambusched94,7
308253,2023-02-10,tt0101540,ambusched94,8
...,...,...,...,...
432901,2019-01-31,tt0093565,anasismael,9
320446,2018-11-20,tt6998518,anasismael,8
387445,2022-03-08,tt0109045,elisereid-29666,10
349192,2021-09-03,tt0081353,elisereid-29666,10


In [25]:
train_df

,date,imdb_id,author,stars
3,2022-07-21,tt0765010,nicktusk-95591,7
4,2022-05-01,tt0765010,peterwixongb,7
6,2022-01-19,tt0765010,kamsitheprince,9
8,2021-09-08,tt0765010,zacharyrivas21,7
9,2021-08-11,tt0765010,SameirAli,7
...,...,...,...,...
450929,2006-12-28,tt0454082,superduperspit,5
450930,2006-12-26,tt0454082,FrightMeter,9
450931,2006-12-25,tt0454082,sackjigler,7
450932,2006-12-25,tt0454082,Nightman85,3


In [27]:
train_df.to_csv("training_set.csv",index=False)
test_df.to_csv("testing_set.csv",index=False)